### Imports
Import the required libraries

In [1]:
# package(s) related to time, space and id
import datetime, time
import platform
import itertools
# Used for mathematical functions
import math             
import logging

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy
import pandas as pd

# spatial libraries 
# import pyproj
# import shapely.geometry
# from simplekml import Kml, Style

# package(s) for data handling
import numpy as np
import matplotlib.pyplot as plt
import tqdm


import plotly.express as px
from plotly.subplots import make_subplots

# OpenTNSim
import opentnsim

# turn on debug messages if want to see what's going on in detail
# logging.basicConfig(level=logging.DEBUG)
# turn off debug messages, only show info and higher level messages
logging.basicConfig(level=logging.INFO)

### Create vessel - add VesselProperties and ConsumesEnergy mixins

In [2]:
# Make your preferred class out of available mix-ins.
TransportResource = type(
    "Vessel",
    (
        opentnsim.core.Identifiable,
        opentnsim.core.Movable,
        opentnsim.core.VesselProperties,  # needed to add vessel properties
        opentnsim.energy.ConsumesEnergy,
        opentnsim.core.ExtraMetadata,
    ),
    {},
)  # needed to calculate resistances

In [3]:
# # Create a dict with all important settings

# data_vessel = {
#     "env": None,
#     "name": 'Vessel M6',
#     "route": None,
#     "geometry": None,
#     "v": None,  # m/s
#     "V_g_ave":3,
#     "V_g_profile":False,
#     "type": None,
#     "B": 9.5,
#     "L": 85,
#     "H_e": None, 
#     "H_f": None, 
#     "T": 2,
#     "safety_margin": 0.3, # for tanker vessel with rocky bed the safety margin is recommended as 0.3 m
#     "h_squat": False, # if consider the ship squatting while moving, set to True, otherwise set to False. Note that here we have disabled h_squat calculation since we regard the water depth h_0 is already reduced by squat effect. This applies to figures 3, 5, 7, 8 and 9.
#     "payload":None,
#     "vessel_type":"Dry_DH", #vessel types: "Container","Dry_SH","Dry_DH","Barge","Tanker". ("Dry_SH" means dry bulk single hull, "Dry_DH" means dry bulk double hull)    
#     "P_installed": 780.0,   
#     "P_tot_given": None, # kW
#     "P_tot_given_profile":False,
#     "bulbous_bow": False, # if a vessel has no bulbous_bow, set to False; otherwise set to True.
#     "sailing_on_power": False,
#     "sailing_upstream":False,
#     "wind_influence": False,
#     "P_hotel_perc": 0,
#     "P_hotel": None, # None: calculate P_hotel from percentage
#     "x": 1,# number of propellers
#     "L_w": 2.0 ,
#     "C_B":0.85, 
#     "C_year": 1961,
# }             



In [4]:
# Create a dict with all important settings

data_vessel = {
    "env": None,
    "name": 'Vessel M6',
    "route": None,
    "geometry": None,
    "v": None,  # m/s
    "V_g_ave":3,
    "V_g_profile":False,
    "type": None,
    "B": 10.5,
    "L": 155.75,
    "H_e": None, 
    "H_f": None, 
    "T": 2,
    "safety_margin": 0.3, # for tanker vessel with rocky bed the safety margin is recommended as 0.3 m
    "h_squat": False, # if consider the ship squatting while moving, set to True, otherwise set to False. Note that here we have disabled h_squat calculation since we regard the water depth h_0 is already reduced by squat effect. This applies to figures 3, 5, 7, 8 and 9.
    "payload":None,
    "vessel_type":"Dry_DH", #vessel types: "Container","Dry_SH","Dry_DH","Barge","Tanker". ("Dry_SH" means dry bulk single hull, "Dry_DH" means dry bulk double hull)    
    "P_installed": 780.0,   
    "P_tot_given": None, # kW
    "P_tot_given_profile":False,
    "bulbous_bow": False, # if a vessel has no bulbous_bow, set to False; otherwise set to True.
    "sailing_on_power": False,
    "sailing_upstream":False,
    "wind_influence": False,
    "P_hotel_perc": 0,
    "P_hotel": None, # None: calculate P_hotel from percentage
    "x": 1,# number of propellers
    "L_w": 2.0 ,
    "C_B":0.85, 
    "C_year": 1961,
}             



In [5]:
data_vessel_i = data_vessel.copy()
vessel = TransportResource(**data_vessel_i)

### Set vessel properties and sailing conditions ( input value )

In [6]:
V_s = np.linspace(2.5, 5, 11)          # ship sailing speeds to water, (m/s)
h_0 = [3,3.3,4.2,5,8]                 # water depths,(m)
T = [2]   # draught


### prepare input matrix for calculation

In [7]:
# prepare the work to be done
# create a list of all combinations
work = list(itertools.product(T, h_0, V_s))

# prepare a list of dictionaries for pandas
rows = []
for item in work:
    row = {"T": item[0], "h_0": item[1], "V_s": item[2]}
    rows.append(row)

# these are all the simulations that we want to run
# convert them to dataframe, so that we can apply a function and monitor progress
work_df = pd.DataFrame(rows)
work_df.tail()

,T,h_0,V_s
50,2,8.0,4.00
51,2,8.0,4.25
52,2,8.0,4.50
53,2,8.0,4.75
54,2,8.0,5.00


### Run OpenTNSim to Calculate resistance, required power and emissions 

In [8]:

results = []

for i, row in tqdm.tqdm(work_df.iterrows()):
    # create a new vessel, like the one above (so that it also has L)
    
    data_vessel_i = data_vessel.copy()
    
    vessel = TransportResource(**data_vessel_i)
    vessel._T = row['T']
    V_s = row['V_s']
    h_0 = row['h_0']
    vessel.calculate_properties() # L is used here in the computation of L_R
    # h_0 = vessel.calculate_h_squat(v = V_s, h_0 = h_0)
    # print(h_0)
    R_f = vessel.calculate_frictional_resistance(V_s, h_0) 
    R_f_one_k1 = vessel.calculate_viscous_resistance()
    R_APP = vessel.calculate_appendage_resistance(V_s)
    R_W = vessel.calculate_wave_resistance(V_s, h_0)
    R_res = vessel.calculate_residual_resistance(V_s, h_0)
    R_T = vessel.calculate_total_resistance(V_s, h_0)
    P_tot = vessel.calculate_total_power_required(V_s, h_0)

    
    result = {}
    result.update(row)
    result['P_installed'] = vessel.P_installed
    result['R_f'] = R_f
    result['R_f_one_k1'] = R_f_one_k1
    result['R_APP'] = R_APP
    result['R_W'] = R_W
    result['R_res'] = R_res
    result['R_T'] = R_T
    result['P_tot'] = P_tot

    results.append(result)

55it [00:00, 782.95it/s]


In [9]:
plot_df = pd.DataFrame(results)


# convert from meters per second to km per hour
ms_to_kmh = 3.6
plot_df['V_s_km'] = plot_df['V_s'] * ms_to_kmh

plot_df.head(50)

,T,h_0,V_s,P_installed,R_f,R_f_one_k1,R_APP,R_W,R_res,R_T,P_tot,V_s_km
0,2.0,3.0,2.50,780.0,12.912992,14.552121,1.614124,3.178560,5.697804,25.042609,"(125.21304484191126, 125.21304484191126, 125.2...",9.0
1,2.0,3.0,2.75,780.0,15.397726,17.352259,1.924716,5.255137,6.816201,31.348313,"(172.41572037843952, 172.41572037843952, 172.4...",9.9
2,2.0,3.0,3.00,780.0,18.083312,20.378743,2.260414,8.020732,8.018848,38.678737,"(232.07242142520874, 232.07242142520874, 232.0...",10.8
3,2.0,3.0,3.25,780.0,20.967480,23.629017,2.620935,11.505298,9.301869,47.057120,"(305.87127841549733, 305.87127841549733, 305.8...",11.7
4,2.0,3.0,3.50,780.0,24.048182,27.100773,3.006023,15.712265,10.661390,56.480451,"(395.36315762251724, 395.36315762251724, 395.3...",12.6
5,2.0,3.0,3.75,780.0,27.323551,30.791905,3.415444,20.624094,12.093536,66.924980,"(501.93734957696796, 501.93734957696796, 501.9...",13.5
6,2.0,3.0,4.00,780.0,30.791876,34.700487,3.848985,26.208511,13.594433,78.352415,"(626.8193231171952, 626.8193231171952, 626.819...",14.4
7,2.0,3.0,4.25,780.0,34.451579,38.824740,4.306447,32.421682,15.160205,90.713074,"(771.0611314480411, 771.0611314480411, 771.061...",15.3
8,2.0,3.0,4.50,780.0,38.301197,43.163015,4.787650,39.221148,16.786977,103.958790,"(935.6291109737796, 935.6291109737796, 780.0)",16.2
9,2.0,3.0,4.75,780.0,42.339368,47.713777,5.292421,46.549144,18.470875,118.026217,"(1121.2490600033843, 1121.2490600033843, 780.0)",17.1


In [10]:
plot_df.tail(25)

,T,h_0,V_s,P_installed,R_f,R_f_one_k1,R_APP,R_W,R_res,R_T,P_tot,V_s_km
30,2.0,4.2,4.50,780.0,34.032867,38.352878,4.254108,39.221148,16.786977,98.615112,"(887.5360078930618, 887.5360078930618, 780.0)",16.2
31,2.0,4.2,4.75,780.0,37.639864,42.417734,4.704983,46.549144,18.470875,112.142736,"(1065.3559942853426, 1065.3559942853426, 780.0)",17.1
32,2.0,4.2,5.00,780.0,41.415825,46.673003,5.176978,54.364369,20.208025,126.422374,"(1264.2237439179603, 1264.2237439179603, 780.0)",18.0
33,2.0,5.0,2.50,780.0,11.098776,12.507615,1.387347,3.178560,5.697804,22.771325,"(113.85662697883282, 113.85662697883282, 113.8...",9.0
34,2.0,5.0,2.75,780.0,13.250281,14.932224,1.656285,5.255137,6.816201,28.659848,"(157.62916306734297, 157.62916306734297, 157.6...",9.9
35,2.0,5.0,3.00,780.0,15.578103,17.555532,1.947263,8.020732,8.018848,35.542375,"(213.25425122959678, 213.25425122959678, 213.2...",10.8
36,2.0,5.0,3.25,780.0,18.080391,20.375451,2.260049,11.505298,9.301869,43.442667,"(282.37733731141844, 282.37733731141844, 282.3...",11.7
37,2.0,5.0,3.50,780.0,20.755468,23.390093,2.594434,15.712265,10.661390,52.358183,"(366.50727835869384, 366.50727835869384, 366.5...",12.6
38,2.0,5.0,3.75,780.0,23.601809,26.597739,2.950226,20.624094,12.093536,62.265595,"(466.9919658931996, 466.9919658931996, 466.991...",13.5
39,2.0,5.0,4.00,780.0,26.618012,29.996808,3.327252,26.208511,13.594433,73.127003,"(585.0160253486625, 585.0160253486625, 585.016...",14.4


In [11]:
data_vessel_i = data_vessel.copy()
vessel = TransportResource(**data_vessel_i)


In [12]:
R=vessel.calculate_total_resistance(v=4, h_0=5)
P = vessel.calculate_total_power_required(v=4, h_0=5)
P

(585.0160253486625, 585.0160253486625, 585.0160253486625)

In [14]:
V_w1 = opentnsim.strategy.power2v(vessel, h_0=5,power_applied=585, upperbound=8)
V_w1

(227.4379266832717, 227.4379266832717, 227.4379266832717) P_given
585 227.4379266832717 357.5620733167283
(1205.0093594811199, 1205.0093594811199, 780.0) P_given
585 1205.0093594811199 -620.0093594811199
(45.87123266685087, 45.87123266685087, 45.87123266685087) P_given
585 45.87123266685087 539.1287673331491
(281.2059800902251, 281.2059800902251, 281.2059800902251) P_given
585 281.2059800902251 303.7940199097749
(533.1340753246928, 533.1340753246928, 533.1340753246928) P_given
585 533.1340753246928 51.865924675307156
(748.6270814284876, 748.6270814284876, 748.6270814284876) P_given
585 748.6270814284876 -163.6270814284876
(553.1559130865867, 553.1559130865867, 553.1559130865867) P_given
585 553.1559130865867 31.84408691341332
(578.8586708508287, 578.8586708508287, 578.8586708508287) P_given
585 578.8586708508287 6.14132914917127
(586.3926238572911, 586.3926238572911, 586.3926238572911) P_given
585 586.3926238572911 -1.3926238572911416
(585.0845695390403, 585.0845695390403, 585.08456953

3.9999684785131517